In [1]:
import pandas as pd
import torch as th
import os
import re
import yfinance as yf
import numpy as np
import warnings
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
warnings.filterwarnings('ignore')
            
import sys; sys.path.insert(0, '..')
from model.models import run_ensemble_strategy, TrainerConfig, Trainer
import data.preprocessing as pp
from utils.utils_analyze import get_price, check_index_dim
from utils.indicators import indicator_list, indicators_stock_stats
from env.BaseEnv import EnvConfig
from env.EnvStock_val import StockEnvValidation
from env.EnvStock_train import StockEnvTrain
from env.EnvStock_trade import StockEnvTrade
from policy.Policies import MlActorCriticPolicy
from config.config import indexes, ticker_list, category_dict, ticker_list_with_dict, tech_tickers
%matplotlib inline
%load_ext autoreload
%autoreload 2

### Do not Run these two cell if you do not want to preprocess data from the begining

In [2]:
START_DATE = '2011-01-01'
SEED = 42069
NORNAMLIZE = True

###  Run these two cell if you do not want to preprocess data from the begining

In [3]:
SAC_PARAMS = {'gamma':0.995,
 'learning_rate':0.016241092434986143,
 'batch_size':64,
 'buffer_size':1000000,
 'learning_starts':10000,
 'gradient_steps':8,
 'ent_coef':'auto',
 'tau':0.005,
 'target_entropy':'auto',
 'policy_kwargs':{'log_std_init': -3.3486909298160947,'net_arch': [256, 256] },
    'use_sde': False,
 'seed':42069,
             "device":"cuda"}


PPO_PARAMS =  {'n_steps': 512, 'batch_size': 64, 'gamma': 0.999, 'learning_rate': 0.03691157097485833, 
                                'ent_coef': 0.08932887565089782, 
                                'clip_range': 0.1, 'n_epochs': 5, 
                                'gae_lambda': 0.9, 'max_grad_norm': 0.8, 'vf_coef': 0.891060944163853, 
                                'policy_kwargs': {'net_arch': [{'pi': [256, 256], 'vf': [256, 256]}], 
              'activation_fn': th.nn.ReLU, 'ortho_init': False}, 'seed': 42069, "device":"cuda"}
DATASET_VERSION = 'pretrainedTrial6'
model = 'reccurent_ppo'
MODEL_NAME = f'jimmy_{model}-v1'
POPULATION = 10
TIME_INTERVAL = 5
START_TRADE = '2011-01-01'
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=[dict(pi=[256, 128, 64], vf=[256, 128])])
TIME_FRAME = 0

In [4]:
winner_params = {'n_steps':8,'batch_size':8,'gamma':0.95,'learning_rate':0.027651081986547765,
 'ent_coef':0.009004220378275581,'clip_range':0.3,'n_epochs':1,'gae_lambda':0.95,'max_grad_norm':0.9,'vf_coef':0.9311987337638572,
 'policy_kwargs':{'net_arch': [{'pi': [256, 256], 'vf': [256, 256]}], 'activation_fn': th.nn.Tanh, 'ortho_init': False},'seed':42069,'device':'cuda'}

In [5]:
params = {'SAC':SAC_PARAMS, 'PPO':PPO_PARAMS}

In [6]:
tconfig = TrainerConfig(start_date=START_TRADE,
                        rebalance_window = 242,
                        validation_window = 242,
                        **{'hparams':winner_params},
                        timesteps=50000,
                       policy_kwargs = policy_kwargs,
                       index_list = indexes)

In [9]:
trainer = Trainer(model, 'MlpLstmPolicy', 
                  StockEnvTrain, StockEnvValidation, StockEnvTrade, 
                  dataset_version=DATASET_VERSION, population=10,
                     config = tconfig, model_name= MODEL_NAME, debug=False)

Total number of stocks:   469


In [10]:
trainer.train(dataset='datasets/downew30.csv', timesteps=50050, 
              load=False, model_to_load='model_to_load', normalize =NORNAMLIZE)

wandb: wandb version 0.13.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


======Model training from:  2011-01-01 to  2016-01-04T00:00:00.000000000
======Training Agents with the population of 10========
Training time  jimmy_reccurent_ppo-v1 :  10.252782766024271  minutes
======jimmy_reccurent_ppo-v1 Validation from:  2016-01-04 00:00:00 to  2016-12-15 00:00:00
-----------------
Total Reward:  1.5654901936650276
Total Trades:  3227
End total asset for validation 1077768.1962370635
Mean Reward: 3.207644689257722
STD reward: 0.00944508255588643
-----------------
Reward for the period is 1.5654901936650276
Agent #0 has better performance for the training period with total reward: 1077768.1962370635
Model destabilized with params:   Creating new params


NameError: name 'sample_ppo_params' is not defined

In [20]:

pretrain_set = pd.read_csv('datasets/pretrain_set3.csv')
pretrain_set['turbulence'] = 0

In [ ]:
pretrain = trainer.pretrain(sn, '2011-05-01', START_TRADE, NORNAMLIZE)

In [67]:
model_to_load = pretrain.split('/')[1]

In [ ]:
trainer.cluster(False, period=365, number_of_clusters=5, stocks_per_cluster=5)

In [60]:
m = pd.read_csv('./datasets/.csv')

TypeError: list indices must be integers or slices, not tuple